In [ ]:
import os
import numpy as np
import keras
from keras.utils.data_utils import get_file
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Model, Sequential
from keras.layers import Input, Flatten, Dense, Dropout, Conv2D, MaxPooling2D
from numpy import asarray
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array


In [ ]:
#Working with 3-channel (RGB) 224x224 images for the VGG16
INPUT_SHAPE = (224, 224, 3)
COARSE_CATEGORIES = ['Religious', 'Residential', 'Commercial', 'Business']
FINE_CATEGORIES = ['Church', 'Mosque', 'Synagogue', 'BuddhistTemple', 'House', 'ApartmentBuilding', 'Mall', 'Supermarket', 'Restaurant', 'OfficeBuilding']
NB_COARSE_CLASSES = 4
NB_FINE_CLASSES = 10

In [ ]:
print(os.listdir("../input"))

In [ ]:
photos = np.load('../input/FinalDatasetPhotos_array.npy', allow_pickle=True)

fine_labels = np.load('../input/FinalDatasetLabels_array.npy', allow_pickle=True)

In [ ]:
#Definining the mapping fine:coarse
fine_to_coarse = {0:0, 1:0, 2:0, 3:0, 4:1, 5:1, 6:2, 7:2, 8:2, 9:3}

In [ ]:
#Importing the weights of the pretrained VGG16
weights_file = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
weights = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5', weights_file, cache_subdir='models')

In [ ]:
#np.random.seed(0)

#Model input
model_input = Input(shape=INPUT_SHAPE, name='image_input')
#Block 1
model = Conv2D(64, (3,3), padding='same', activation='relu', name='block1_conv1')(model_input)
model = BatchNormalization()(model)
model = Conv2D(64, (3,3), padding='same', activation='relu', name='block1_conv2')(model)
model = BatchNormalization()(model)
model = MaxPooling2D((2,2), strides=(2,2), name='block1_pool')(model)
#Block 2
model = Conv2D(128, (3,3), padding='same', activation='relu', name='block2_conv1')(model)
model = BatchNormalization()(model)
model = Conv2D(128, (3,3), padding='same', activation='relu', name='block2_conv2')(model)
model = BatchNormalization()(model)
model = MaxPooling2D((2,2), strides=(2,2), name='block2_pool')(model)

#Block 3
model = Conv2D(256, (3,3), padding='same', activation='relu', name='block3_conv1')(model)
model = BatchNormalization()(model)
model = Conv2D(256, (3,3), padding='same', activation='relu', name='block3_conv2')(model)
model = BatchNormalization()(model)
model = Conv2D(256, (3,3), padding='same', activation='relu', name='block3_conv3')(model)
model = BatchNormalization()(model)
model = MaxPooling2D((2,2), strides=(2,2), name='block3_pool')(model)

#Here the normal VGG16 architecture is altered to add a branch for coarse category classification
#Coarse branch: predicts a building's general function (Business, Commercial, Residential, Religious)
coarse = Flatten(name='c_flatten')(model)
coarse = Dense(256, activation='relu', name='c_dense1')(coarse)
coarse = BatchNormalization()(coarse)
coarse = Dropout(0.5)(coarse)
coarse = Dense(256, activation='relu', name='c_dense2')(coarse)
coarse = BatchNormalization()(coarse)
coarse = Dropout(0.5)(coarse)
coarse_pred = Dense(NB_COARSE_CLASSES, activation='softmax', name='coarse_prediction')(coarse)


#Block 4
model = Conv2D(512, (3,3), padding='same', activation='relu', name='block4_conv1')(model)
model = BatchNormalization()(model)
model = Conv2D(512, (3,3), padding='same', activation='relu', name='block4_conv2')(model)
model = BatchNormalization()(model)
model = Conv2D(512, (3,3), padding='same', activation='relu', name='block4_conv3')(model)
model = BatchNormalization()(model)
model = MaxPooling2D((2,2), strides=(2,2), name='block4_pool')(model)

#Block 5
model = Conv2D(512, (3,3), padding='same', activation='relu', name='block5_conv1')(model)
model = BatchNormalization()(model)
model = Conv2D(512, (3,3), padding='same', activation='relu', name='block5_conv2')(model)
model = BatchNormalization()(model)
model = Conv2D(512, (3,3), padding='same', activation='relu', name='block5_conv3')(model)
model = BatchNormalization()(model)
model = MaxPooling2D((2,2), strides=(2,2), name='block5_pool')(model)

#Fine branch: predicts a building's specific function (1 of the 10 categories: Church, Mosque, House, Office Building, Mall, etc.)
model = Flatten(name='flatten')(model)
model = Dense(512, activation='relu', kernel_initializer='he_uniform', name='dense1')(model)
model = BatchNormalization()(model)
model = Dropout(0.5)(model)
model = Dense(256, activation='relu', kernel_initializer='he_uniform', name='dense2')(model)
model = BatchNormalization()(model)
model = Dropout(0.2)(model)
coarse_pred = Dense(NB_COARSE_CLASSES, activation='softmax', name='coarse_prediction')(model)
fine_pred = Dense(NB_FINE_CLASSES, activation='softmax', name='fine_pred')(model)


In [ ]:
#loss_w1: Loss weight for the coarse branch
#loss_w2: Loss weight for the fine branch 

class ModifyLossWeights(keras.callbacks.Callback):
    def __init__(self, loss_w1, loss_w2):
        self.loss_w1 = loss_w1
        self.loss_w2 = loss_w2
    # customize your behavior
    def on_epoch_end(self, epoch, logs={}):
        if epoch == 15:
            keras.backend.set_value(self.loss_w1, 0.5)
            keras.backend.set_value(self.loss_w2, 0.5)
        if epoch == 25:
            keras.backend.set_value(self.loss_w1, 0)
            keras.backend.set_value(self.loss_w2, 1)

In [ ]:
loss_w1 = keras.backend.variable(value=0.3, dtype="float32", name="coarse_loss_weight") 
loss_w2 = keras.backend.variable(value=0.7, dtype="float32", name="fine_loss_weight") 

#Preparing the callback:
loss_w_modif = ModifyLossWeights(loss_w1, loss_w2)

bcnn_losses = {"coarse_prediction": "categorical_crossentropy",	"fine_pred": "categorical_crossentropy"}

In [ ]:
model = Model(inputs=model_input, outputs=[coarse_pred, fine_pred])   
model.load_weights(weights, by_name=True)
opt = SGD(lr=0.0001, momentum=0.9)
model.compile(optimizer=opt, loss=bcnn_losses, metrics=['accuracy'], loss_weights=[loss_w1, loss_w2])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(photos, fine_labels, test_size=0.2, 
                                               random_state = np.random.randint(1,356, 1)[0])
X_train = X_train/255
X_test = X_test/255
#Creating empty matrices for the coarse labels of the train and test sets with the appropriate shapes
y_coarse_train = np.zeros(y_train.shape[0]).astype("float32")
y_coarse_test = np.zeros(y_test.shape[0]).astype("float32")

#Filling up the matrices accordingly by putting 1 in the column corresponding to the correct coarse class
for i in range(y_coarse_train.shape[0]):
  y_coarse_train[i] = fine_to_coarse[y_train[i]]
for i in range(y_coarse_test.shape[0]):
  y_coarse_test[i] = fine_to_coarse[y_test[i]]
    
y_test = tf.keras.utils.to_categorical(y_test, num_classes = NB_FINE_CLASSES)
y_train = tf.keras.utils.to_categorical(y_train, num_classes = NB_FINE_CLASSES)
y_coarse_test = tf.keras.utils.to_categorical(y_coarse_test, num_classes = NB_COARSE_CLASSES)
y_coarse_train = tf.keras.utils.to_categorical(y_coarse_train, num_classes = NB_COARSE_CLASSES)
#y_merged_test = np.concatenate((y_coarse_test, y_test), axis = 1)
#y_merged_train = np.concatenate((y_coarse_train, y_train), axis = 1 )

#callbacks=[loss_w_modif]
   
history_model = model.fit(X_train, [y_coarse_train,y_train], batch_size=50, epochs=50, callbacks=[loss_w_modif], verbose=1, validation_split=0.1)  
print("Val Score: ", model.evaluate(X_test, [y_coarse_test, y_test]))

In [ ]:
fig = plt.plot(history_model.history["coarse_prediction_accuracy"],label = "Training - Coarse", color='blue')
plt.plot(history_model.history["val_coarse_prediction_accuracy"],label = "Testing - Coarse", color='green')
plt.plot(history_model.history["fine_pred_accuracy"],label = "Training - Fine", color='brown')
plt.plot(history_model.history["val_fine_pred_accuracy"],label = "Testing - Fine", color='orange')
plt.legend(loc='upper left')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy per epoch")
plt.show()

In [ ]:
plt.plot(history_model.history["coarse_prediction_loss"],label = "Training - Coarse", color='blue')
plt.plot(history_model.history["val_coarse_prediction_loss"],label = "Testing - Coarse", color='green')
plt.plot(history_model.history["fine_pred_loss"],label = "Training - Fine", color='brown')
plt.plot(history_model.history["val_fine_pred_loss"],label = "Testing - Fine", color='orange')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

In [ ]:
from keras.utils.layer_utils import count_params

trainable_count = count_params(model.trainable_weights)
non_trainable_count = count_params(model.non_trainable_weights)
print('Total params: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable params: {:,}'.format(trainable_count))
print('Non-trainable params: {:,}'.format(non_trainable_count))